In [1]:
from probttrtypes import Type, BType, PConstraint, Pred, PType, Possibility, RecType, Fun
from utils import show, show_latex
from records import Rec

## Judgement

In [2]:
T = Type()
T.judge('a',.5)
show(T.query('a'))

'0.5'

## Basic types

In [3]:
Ind = BType('Ind')
Ind.judge('a',1)
show(Ind.query('a'))

'1.0'

An alternative way of judging for probability `1`.

In [4]:
Ind.judge('b')
show(Ind.query('b'))

'1.0'

## Witness conditions for types

In [5]:
def RealClassifier(n):
    if isinstance(n,float):
        return PConstraint(1)
    else:
        return PConstraint(0)
Real = Type('Real')
Real.learn_witness_condition(RealClassifier)
show(Real.query(.6))

'1.0'

In [6]:
show(Real.query('a'))

'0.0'

## Intensionality

In [7]:
T1 = Type()
T2 = Type()
T1.judge('a',.4)
T1.judge('b',.6)
T2.judge('b',.6)
T2.judge('a',.4)

print(show(T1.witness_cache))
print(show(T2.witness_cache))
print(set(show(list(zip(T1.witness_cache[0],T1.witness_cache[1])))) == set(show(list(zip(T2.witness_cache[0],T2.witness_cache[1])))))
print(T1==T2)

([a, b], [0.4, 0.6])
([b, a], [0.6, 0.4])
True
False


## Ptypes

In [8]:
run = Pred('run',[Ind])
hug = Pred('hug',[Ind,Ind])
Ind.judge('b')
Ind.judge('d')
run_d = PType(run,['d'])
hug_b_d = PType(hug,['b','d'])
run_d.judge('e1',.7)
hug_b_d.judge('e2',0)

print(show(run_d.query('e1')))
print(show(hug_b_d.query('e2')))
print(show(hug_b_d.query('e1')))

0.7
0.0
<=1.0


## Possibilities (models)

In [9]:
M1 = Possibility('M1')

Ind.in_poss(M1)
run_d.in_poss(M1)
hug_b_d.in_poss(M1)

print(show(M1))


M1:
_____________________________________________
Ind: [(a, 1.0), (b, 1.0), (d, 1.0)]
run(d): [(e1, 0.7)]
hug(b, d): [(e2, 0.0)]
_____________________________________________



In [10]:
M2 = Possibility('M2')

BType('Ind',M2).judge('c')
PType(run,['c'],M2).judge('e1',.9)

print(show(M2))


M2:
_____________________________________________
Ind: [(c, 1.0)]
run(c): [(e1, 0.9)]
_____________________________________________



## Witness conditions associated with ptypes

In [11]:
left = Pred('left',[Ind,Ind])
right = Pred('right',[Ind,Ind])
left.learn_witness_fun(lambda args: PType(right,[args[1],args[0]]))
right.learn_witness_fun(lambda args: PType(left,[args[1],args[0]]))
left_a_b = PType(left,['a','b'])
right_b_a = PType(right,['b','a'])
M = Possibility('M')
right_b_a.in_poss(M).judge('s1',.6)
left_a_b.in_poss(M).judge('s2',.7)
print(show(M))
print(show(left_a_b.in_poss(M).query('s1')))
print(show(right_b_a.in_poss(M).query('s2')))
print(show(M))


M:
_____________________________________________
right(b, a): [(s1, 0.6)]
left(a, b): [(s2, 0.7)]
_____________________________________________

0.6
0.7

M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.7)]
left(a, b): [(s2, 0.7), (s1, 0.6)]
_____________________________________________



## Record Types

In [12]:
boy = Pred('boy',[Ind])
dog = Pred('dog',[Ind])
a_boy_hugs_a_dog = RecType({'x':Ind,
                            'c_boy':(Fun('v',Ind,PType(boy,['v'])), ['x']),
                            'y':Ind,
                            'c_dog':(Fun('v',Ind,PType(dog,['v'])), ['y']),
                            'e':(Fun('v1',Ind,Fun('v2',Ind, PType(hug,['v1','v2']))), 
                                     ['x','y'])})
print(show(a_boy_hugs_a_dog))
show_latex(a_boy_hugs_a_dog)

{x : Ind, c_boy : (lambda v:Ind . boy(v), [x]), y : Ind, c_dog : (lambda v:Ind . dog(v), [y]), e : (lambda v1:Ind . lambda v2:Ind . hug(v1, v2), [x, y])}


<IPython.core.display.Latex object>

In [13]:
M3 = Possibility('M3')
Ind3 = BType('Ind',M3)
Ind3.judge('a')
Ind3.judge('b')
PType(boy,['a']).in_poss(M3).judge('s1',.9)
PType(dog,['b']).in_poss(M3).judge('s2',.95)
PType(hug,['a','b']).in_poss(M3).judge('s3',.7)

print(show(M3))


M3:
_____________________________________________
Ind: [(a, 1.0), (b, 1.0)]
boy(a): [(s1, 0.9)]
dog(b): [(s2, 0.95)]
hug(a, b): [(s3, 0.7)]
_____________________________________________



In [14]:
r1 = Rec({'x':'a','c_boy':'s1','y':'b','c_dog':'s2','e':'s3'})

print(show(r1))
show_latex(r1)

{x = a, c_boy = s1, y = b, c_dog = s2, e = s3}


<IPython.core.display.Latex object>

In [15]:
show(a_boy_hugs_a_dog.in_poss(M3).query(r1))

'0.5984999999999999'

In [16]:
1*1*.9*.95*.7

0.5984999999999999

## String types

_Not yet implemented_